---
title: C++ const修饰成员函数
tags: 小书匠,c++,const
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# C++ const修饰成员函数

## const 修饰成员函数的原理

**const 修饰成员函数实际上是修饰 this。**

默认情况下 `this` 指针的类型是指向类类型非常量版本的常量指针。具体来说就是：`StrBlob* const`（StrBlob 是一个类），这也就意味着，`this` 指针无法指向一个 `const StrBlob` 对象，也就无法在 `const StrBlob` 对象上使用普通的成员函数。

所以我们想让 `this` 变为指向常量的常量指针，C++ 中的做法是在普通成员函数的参数列表后面加上 const 关键字，这样的成员函数叫做常量成员函数。此时 `this` 的类型相当于 `const StrBlob* const`

## const 修饰成员函数的原则

对于 const 修饰成员函数，有下面几条规则

1. const是函数类型的一部分，**在实现部分也要带该关键字**。
2. const关键字可以用于对重载函数的区分。
3. 常成员函数不能更新类的成员变量，也不能调用该类中没有用const修饰的成员函数，只能调用常成员函数。
4. const 对象只能调用 const成员函数，而非常量对象既可以调用 const 成员函数，也可以调用非常成员函数。但是如果有重载的非常成员函数则会调用非常成员函数。

### const 对象只能调用 const 成员函数

In [14]:
#include<iostream>  
   
class Test  
{  
    private:
        int x;  
    public:  
        Test (int i):x(i) { }  
        void fun()  
        {  
            std::cout << "fun() called " << std::endl;  
        }  
};  

In [15]:
Test t1 (10);  
t1.fun();  
// const Test t2 (20);  
// t2.fun();  // error: member function 'fun' not viable: 'this' argument has type 'const Test', but function is not marked const

fun() called 


### 非 const 对象可以调用 const 函数

In [20]:
#include<iostream>  
   
class Test  
{  
    private:
        int x;  
    public:  
        Test (int i):x(i) { }  
        void fun() const  
        {  
            std::cout << "fun() const called " << std::endl;  
        }  
};  

In [17]:
Test t1 (10);  
const Test t2 (20);  
t1.fun();  // non-const object calls const member function
t2.fun();  // const object calls const member function

fun() const called 
fun() const called 


### 如果一个函数同时重载了 const 版本和非 const版本，那么 const 对象调用 const 版本，非 const 对象调用非 const 版本

In [21]:
#include<iostream>  
   
class Test  
{  
    private:
        int x;  
    public:  
        Test (int i):x(i) { }  
        void fun() const  
        {  
            std::cout << "fun() const called " << std::endl;  
        }  
        void fun()  
        {  
            std::cout << "fun() called " << std::endl;  
        }  
};  

In [22]:
Test t1 (10);  
const Test t2 (20);  
t1.fun();  // non-const object calls non-const member function
t2.fun();  // const object calls const member function

fun() called 
fun() const called 


## const 修饰成员函数的使用原则

const 关键字给予了我们更加丰富的表达方式，让我们可以区别出 const 对象和 非 const 对象。

1. 对于某些函数，如果我们想区分 const 对象和非const 对象，并让他们提供不同的表现时，我们可以编写这个函数的 const 版本和非 const 版本，如下面的例子

In [48]:
#include <iostream>
class People
{
    private:
        std::string name;
        int age_;
    public:
        People(int age=0): age_(age) {}
        int& age() { return age_;}
        const int& age() const { return age_;}
};

In [53]:
People p1;
std::cout << "Before change: " << p1.age() << std::endl;
p1.age() = 1;
std::cout << "After change: " << p1.age() << std::endl;

Before change: 0
After change: 1


In [54]:
const People p2;
std::cout << "Can't change age_ via const function: " << p2.age() << std::endl;
// p2.age = 2; // 报错

Can't change age_ via const function: 0


@0x10785fc30

这个例子中，我们希望 People 这个类，如果是通过非 const 对象来调用 `age()` 时，我们返回一个可以修改的引用。而通过 const 对象来调用 `age()` 时，我们返回一个不能修改的引用。

2. 如果对于某个函数，我们不想区分 const 和 非 const 对象的表现的时候，我们可以只编写这个函数的 const 版本而省略非 const 版本（因为 const 对象和 非 const 对象都可以调用 const 版本）

In [55]:
#include <iostream>
class People
{
    private:
        std::string name;
        int age_;
    public:
        People(int age=0): age_(age) {}
        void show() const { std::cout << age_ << "\n"; }
};

In [59]:
People p1(1);
const People p2(2);
p1.show();
p2.show();

1
2


# References
- [const 成员函数与基于 const 的重载 - xinze - 博客园](https://www.cnblogs.com/LuoboLiam/p/13810797.html)
- [C++中const用于函数重载 - 青儿哥哥 - 博客园](https://www.cnblogs.com/qingergege/p/7609533.html)